In [14]:
import numpy as np
import glob
import music21
import random
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Embedding, Bidirectional, BatchNormalization
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.utils import to_categorical
import pickle
from keras.layers import GRU


In [27]:
SEQUENCE_LENGTH = 80
EPOCHS = 50
BATCH_SIZE = 64
GENERATED_LENGTH = 300
MIDI_PATH = "midi_files/*.mid"
OUTPUT_FILE = "local_files/generated.mid"

In [18]:
# -----------------------------
# STEP 2: Data Preprocessing
# -----------------------------
def parse_midi_files(midi_path):
    """Extract notes & chords from MIDI files"""
    notes = []
    for file in glob.glob(midi_path):
        midi = music21.converter.parse(file)
        notes_to_parse = None

        try:
            s2 = music21.instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[0].recurse()
        except:
            notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, music21.note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, music21.chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))
    return notes

notes = parse_midi_files(MIDI_PATH)
print(f"Total notes extracted: {len(notes)}")

Total notes extracted: 4993


In [20]:
# Saving notes for future use as a pickle file
with open('midi_files/input_notes.pkl', 'wb') as filepath:
    pickle.dump(notes, filepath)

In [21]:
# Mapping notes to integers
unique_notes = sorted(set(notes))
note_to_int = {note: i for i, note in enumerate(unique_notes)}
int_to_note = {i: note for i, note in enumerate(unique_notes)}


In [22]:
# Sequence preparation
network_input, network_output = [], []
for i in range(len(notes) - SEQUENCE_LENGTH):
    seq_in = notes[i:i+SEQUENCE_LENGTH]
    seq_out = notes[i+SEQUENCE_LENGTH]
    network_input.append([note_to_int[ch] for ch in seq_in])
    network_output.append(note_to_int[seq_out])

X = np.array(network_input)
y = to_categorical(network_output, num_classes=len(unique_notes))

print("Shape of Input:", X.shape)
print("Shape of Output:", y.shape)

Shape of Input: (4913, 80)
Shape of Output: (4913, 153)


In [23]:

# -----------------------------
# STEP 3: Advanced Model
# -----------------------------
model = Sequential([
    Embedding(len(unique_notes), 100, input_length=SEQUENCE_LENGTH),  # Embeddings for notes
    Bidirectional(GRU(256, return_sequences=True)),
    Dropout(0.3),
    BatchNormalization(),
    Bidirectional(GRU(256, return_sequences=True)),
    Dropout(0.3),
    BatchNormalization(),
    GRU(256),
    Dense(256, activation="relu"),
    Dropout(0.3),
    Dense(len(unique_notes), activation="softmax")
])

model.compile(loss="categorical_crossentropy", optimizer="adam")

# -----------------------------
# STEP 4: Training with Callbacks
# -----------------------------
checkpoint = ModelCheckpoint(
    "best_model.h5",
    monitor="loss",
    save_best_only=True,
    mode="min"
)

early_stop = EarlyStopping(
    monitor="loss",
    patience=10,
    restore_best_weights=True
)

reduce_lr = ReduceLROnPlateau(
    monitor="loss",
    factor=0.5,
    patience=5,
    verbose=1
)

history = model.fit(
    X, y,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks=[checkpoint, early_stop, reduce_lr]
)



Epoch 1/50
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - loss: 4.1421

77/77 ━━━━━━━━━━━━━━━━━━━━ 137s 2s/step - loss: 4.1367 - learning_rate: 0.0010
Epoch 2/50
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - loss: 3.0044

77/77 ━━━━━━━━━━━━━━━━━━━━ 113s 1s/step - loss: 3.0037 - learning_rate: 0.0010
Epoch 3/50
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - loss: 2.5568

77/77 ━━━━━━━━━━━━━━━━━━━━ 117s 2s/step - loss: 2.5573 - learning_rate: 0.0010
Epoch 4/50
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - loss: 2.3145

77/77 ━━━━━━━━━━━━━━━━━━━━ 100s 1s/step - loss: 2.3145 - learning_rate: 0.0010
Epoch 5/50
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - loss: 2.0906

...

77/77 ━━━━━━━━━━━━━━━━━━━━ 115s 1s/step - loss: 0.1460 - learning_rate: 5.0000e-04
Epoch 47/50
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - loss: 0.0965

77/77 ━━━━━━━━━━━━━━━━━━━━ 121s 2s/step - loss: 0.0967 - learning_rate: 5.0000e-04
Epoch 48/50
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - loss: 0.0961

77/77 ━━━━━━━━━━━━━━━━━━━━ 122s 2s/step - loss: 0.0961 - learning_rate: 5.0000e-04
Epoch 49/50
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - loss: 0.0837

77/77 ━━━━━━━━━━━━━━━━━━━━ 124s 2s/step - loss: 0.0838 - learning_rate: 5.0000e-04
Epoch 50/50
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - loss: 0.0874

77/77 ━━━━━━━━━━━━━━━━━━━━ 121s 2s/step - loss: 0.0875 - learning_rate: 5.0000e-04


In [25]:
# saving the model
model.save('local_files/music_model.keras')

In [26]:
# -----------------------------
# STEP 5: Advanced Music Generation
# -----------------------------
def sample_with_temperature(preds, temperature=1.0):
    """Sample an index from a probability array with temperature scaling"""
    preds = np.asarray(preds).astype("float64")
    preds = np.log(preds + 1e-8) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def generate_notes(model, seed, length=200, temperature=1.0):
    """Generate notes from trained model"""
    generated = []
    pattern = seed.copy()

    for i in range(length):
        prediction_input = np.array(pattern).reshape(1, -1)
        prediction = model.predict(prediction_input, verbose=0)[0]
        index = sample_with_temperature(prediction, temperature)
        result = int_to_note[index]
        generated.append(result)
        pattern.append(index)
        pattern = pattern[1:]
    return generated

# Pick a random seed
start_index = np.random.randint(0, len(X)-1)
seed = network_input[start_index]
generated_notes = generate_notes(model, seed, length=GENERATED_LENGTH, temperature=0.8)


In [29]:
# -----------------------------
# STEP 6: Convert back to MIDI
# -----------------------------
from music21 import stream, note, chord, instrument

offset = 0
output_notes = []

for pattern in generated_notes:
    if '.' in pattern:  # chord
        notes_in_chord = pattern.split('.')
        chord_notes = []
        for n in notes_in_chord:
            try:
                # Convert pitch class to note name (C=0, C#=1, ..., B=11)
                pitch_class = int(n)
                note_name = music21.pitch.Pitch(pitch_class).name
                chord_note = note.Note(note_name + '4')  # default octave 4
            except Exception:
                chord_note = note.Note('C4')  # fallback to C4 if conversion fails
            chord_note.storedInstrument = instrument.Piano()
            chord_notes.append(chord_note)
        new_chord = chord.Chord(chord_notes)
        new_chord.offset = offset
        output_notes.append(new_chord)
    else:  # single note
        try:
            # Try to interpret as pitch class
            pitch_class = int(pattern)
            note_name = music21.pitch.Pitch(pitch_class).name
            new_note = note.Note(note_name + '4')
        except Exception:
            # If not an integer, try as note name directly
            try:
                new_note = note.Note(pattern)
            except Exception:
                new_note = note.Note('C4')
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)
    offset += 0.5

midi_stream = stream.Stream(output_notes)
midi_stream.write("midi", fp=OUTPUT_FILE)

print(f"✅ Generated music saved to {OUTPUT_FILE}")

✅ Generated music saved to local_files/generated.mid
